<a href="https://colab.research.google.com/github/mohammedterry/NLP_for_ML/blob/master/NLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gluon NLP

In [2]:
!pip3 install mxnet
!pip3 install gluonnlp

    100% |████████████████████████████████| 28.4MB 912kB/s 
    100% |████████████████████████████████| 61kB 21.1MB/s 
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 1.0.0 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4


    100% |████████████████████████████████| 194kB 6.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/19/c4/f4/70dc14c8b01db12cd4f88c6f71128bb88de545a7690e192ce0
Successfully built gluonnlp


In [3]:
import mxnet as mx
import gluonnlp as nlp
ctx = mx.cpu()
lm_model, vocab = nlp.model.get_model(name='awd_lstm_lm_1150', dataset_name='wikitext-2', pretrained=True, ctx=ctx)

Vocab file is not found. Downloading.


In [0]:
scorer = nlp.model.BeamSearchScorer(alpha=0, K=5, from_logits=False)

In [0]:
class LMDecoder(object):
    def __init__(self, model):
        self._model = model
    def __call__(self, inputs, states):
        outputs, states = self._model(mx.nd.expand_dims(inputs, axis=0), states)
        return outputs[0], states
    def state_info(self, *arg, **kwargs):
        return self._model.state_info(*arg, **kwargs)
decoder = LMDecoder(lm_model)

In [0]:
eos_id = vocab['.']
beam_sampler = nlp.model.BeamSearchSampler(beam_size=5,decoder=decoder,eos_id=eos_id,scorer=scorer,max_length=20)

In [0]:
bos = 'I like the Hulk but Im actually more of a DC fan'.split()
bos_ids = [vocab[ele] for ele in bos]
begin_states = lm_model.begin_state(batch_size=1, ctx=ctx)
if len(bos_ids) > 1:
    _, begin_states = lm_model(mx.nd.expand_dims(mx.nd.array(bos_ids[:-1]), axis=1),begin_states)
inputs = mx.nd.full(shape=(1,), ctx=ctx, val=bos_ids[-1])

In [0]:
def generate_sequences(sampler, inputs, begin_states, num_print_outcomes):
    samples, scores, valid_lengths = sampler(inputs, begin_states)
    samples = samples[0].asnumpy()
    scores = scores[0].asnumpy()
    valid_lengths = valid_lengths[0].asnumpy()
    print('Generation Result:')
    for i in range(num_print_outcomes):
        sentence = bos[:-1]
        for ele in samples[i][:valid_lengths[i]]:
            sentence.append(vocab.idx_to_token[ele])
        print([' '.join(sentence), scores[i]])

In [19]:
generate_sequences(beam_sampler, inputs, begin_states, 5)

Generation Result:
['I like the Hulk but Im actually more of a DC fan .', -0.94576716]
['I like the Hulk but Im actually more of a DC fan , but it was not until the end of the 20th century that it was not until the end of .', -27.151623]
['I like the Hulk but Im actually more of a DC fan , but it was not until the end of the 20th century that it was not the same .', -27.202784]
['I like the Hulk but Im actually more of a DC fan , but it was not until the end of the 20th century that it was not until the early 1990s .', -29.516235]
['I like the Hulk but Im actually more of a DC fan , but it was not until the end of the 20th century that it was not until the early 1980s .', -29.629818]


In [0]:
seq_sampler = nlp.model.SequenceSampler(beam_size=5,decoder=decoder,eos_id=eos_id,max_length=100,temperature=0.97)

In [21]:
generate_sequences(seq_sampler, inputs, begin_states, 5)

Generation Result:
['I like the Hulk but Im actually more of a DC fan , throughout Japanese history .', -23.0509]
['I like the Hulk but Im actually more of a DC fan like the prime minister , whose mark is " wild for " ; casualty and reptile .', -95.719086]
['I like the Hulk but Im actually more of a DC fan .', -0.94576716]
['I like the Hulk but Im actually more of a DC fan pays off the ground on department <unk> , forcing several or more errors to come drawings , from focusing on <unk> techniques by their own special colors ( and often take down all individual equipment , screenplay and vocals ) .', -213.31241]
['I like the Hulk but Im actually more of a DC fan , he also used to desperately submit to the knowledge of the social story of the Māori .', -77.988]


# Generate using OpenAI implementation

In [22]:
!git clone https://github.com/openai/gpt-2
import os
os.chdir('gpt-2')
!sh download_model.sh 117M
!pip3 install -r requirements.txt

Cloning into 'gpt-2'...
remote: Enumerating objects: 158, done.
remote: Total 158 (delta 0), reused 0 (delta 0), pack-reused 158
Receiving objects: 100% (158/158), 4.35 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (82/82), done.
sh: 0: Can't open download_model.sh
    100% |████████████████████████████████| 604kB 21.4MB/s 
    100% |████████████████████████████████| 51kB 20.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
spacy 2.0.18 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.

In [25]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 615kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 46.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 616kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:08, 61.7Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.50Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 41.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 43.1Mit/s]                                                       


In [26]:
!python3 src/interactive_conditional_samples.py --nsamples=3 --length=175 --top_k 40 --temperature 0.7

2019-03-09 22:14:09.679025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-09 22:14:09.682018: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2829fa0 executing computations on platform Host. Devices:
2019-03-09 22:14:09.682079: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Model prompt >>> I like the Hulk but Im actually more of a DC fan 
======================================== SAMPLE 1 ========================================
__________________

"Fifty-nine percent of what you see is what you'll see now."

"The greatest thing in the world isn't where we're going to end up but where we're gonna find 